# 效能指標-預測最低價格

## 預測最低價格

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# 讀取原始數據
file_path = '/Users/yuchingchen/Documents/專題/cleaned_data/short_flight.csv'
data = pd.read_csv(file_path)

# 分離經濟艙與商務艙的數據
economy_class_data = data[data["艙等"] == "經濟艙"].copy()
business_class_data = data[data["艙等"] == "商務艙"].copy()

# One-hot encoding（指定分類欄位）
categorical_cols = ['星期', '出發時段', '出發機場代號', '抵達時段', '抵達機場代號',
                    '航空公司', '航空聯盟', '機型分類', '是否過夜', '是否為平日', '機型', '假期', 'Region']
economy_class_data = pd.get_dummies(economy_class_data, columns=categorical_cols, drop_first=True)
business_class_data = pd.get_dummies(business_class_data, columns=categorical_cols, drop_first=True)

# 取兩個數據集共同擁有的欄位
common_cols = list(set(economy_class_data.columns) & set(business_class_data.columns))
economy_class_data = economy_class_data[common_cols]
business_class_data = business_class_data[common_cols]

# 標準化數值變數
num_cols = ['飛行時間_分鐘', '經濟指標', '機場指標', 'competing_flights']
scaler = StandardScaler()
economy_class_data[num_cols] = scaler.fit_transform(economy_class_data[num_cols])
business_class_data[num_cols] = scaler.transform(business_class_data[num_cols])

# 選取建模特徵（依據欄位名稱包含特定關鍵字）
target_keywords = ['出發時段_', '出發機場代號_', '抵達時段_', '航空聯盟_', '機型分類_', '假期_', '是否為平日_']
economy_dummy_cols = [col for col in economy_class_data.columns if any(keyword in col for keyword in target_keywords)]
business_dummy_cols = [col for col in business_class_data.columns if any(keyword in col for keyword in target_keywords)]
economy_feature_cols = economy_dummy_cols + num_cols
business_feature_cols = business_dummy_cols + num_cols

# 設定隨機種子
seed = 123

# 定義通用訓練與預測函數，輸入模型、資料與模型標籤
def train_and_predict_model(model, X_train, X_test, y_train, y_test, model_label):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"📌 {model_label} 測試集 MSE: {mse:.4f}")
    print(f"📌 {model_label} 測試集 R²: {r2:.4f}")
    return y_pred

# 定義三個模型
rf_model = RandomForestRegressor(n_estimators=100, random_state=seed, n_jobs=-1)
svr_model = SVR()
xgb_model = XGBRegressor(n_estimators=100, random_state=seed, n_jobs=-1)

# === 經濟艙預測 ===
print("====== 經濟艙模型訓練 ======")
X_economy = economy_class_data[economy_feature_cols]
y_economy = economy_class_data["最低價格_log"]

# 切分資料（同一測試集用於所有模型比較）
X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(X_economy, y_economy, test_size=0.3, random_state=seed)

# 分別使用 RF、SVR、XGB 模型進行預測
economy_pred_rf = train_and_predict_model(rf_model, X_train_e, X_test_e, y_train_e, y_test_e, "RF")
economy_pred_svr = train_and_predict_model(svr_model, X_train_e, X_test_e, y_train_e, y_test_e, "SVR")
economy_pred_xgb = train_and_predict_model(xgb_model, X_train_e, X_test_e, y_train_e, y_test_e, "XGB")

# 建立預測結果資料（取測試集索引），並將 log 轉換回原始價格
economy_results = economy_class_data.loc[X_test_e.index].copy()
economy_results["預測最低價格_rf"] = np.exp(economy_pred_rf)
economy_results["預測最低價格_svr"] = np.exp(economy_pred_svr)
economy_results["預測最低價格_xgb"] = np.exp(economy_pred_xgb)

# === 商務艙預測 ===
print("\n====== 商務艙模型訓練 ======")
X_business = business_class_data[business_feature_cols]
y_business = business_class_data["最低價格_log"]

# 切分資料
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_business, y_business, test_size=0.3, random_state=seed)

# 分別使用 RF、SVR、XGB 模型進行預測
business_pred_rf = train_and_predict_model(rf_model, X_train_b, X_test_b, y_train_b, y_test_b, "RF")
business_pred_svr = train_and_predict_model(svr_model, X_train_b, X_test_b, y_train_b, y_test_b, "SVR")
business_pred_xgb = train_and_predict_model(xgb_model, X_train_b, X_test_b, y_train_b, y_test_b, "XGB")

# 建立預測結果資料（取測試集索引）
business_results = business_class_data.loc[X_test_b.index].copy()
business_results["預測最低價格_rf"] = np.exp(business_pred_rf)
business_results["預測最低價格_svr"] = np.exp(business_pred_svr)
business_results["預測最低價格_xgb"] = np.exp(business_pred_xgb)

# === 合併預測結果 ===
# 將經濟艙與商務艙的預測結果合併回原始資料
predicted_data = pd.concat([economy_results, business_results]).sort_index()

# 如果原始資料除了經濟艙與商務艙還有其他資料，
# 可用原始 data 合併預測結果，未預測的欄位保持原狀：
data_updated = data.merge(predicted_data[['預測最低價格_rf','預測最低價格_svr','預測最低價格_xgb']],
                          left_index=True, right_index=True, how='left')

# 儲存預測結果到新檔案
output_path = '/Users/yuchingchen/Documents/專題/model/data/short_flight_predict.csv'
data_updated.to_csv(output_path, index=False)

# 額外儲存經濟艙與商務艙的預測結果
economy_output = '/Users/yuchingchen/Documents/專題/model/efficacy_data/short_economy_predicted.csv'
business_output = '/Users/yuchingchen/Documents/專題/model/efficacy_data/short_business_predicted.csv'
economy_results.to_csv(economy_output, index=False)
business_results.to_csv(business_output, index=False)

print("\n✅ 預測完成，合併結果與分別的艙別結果皆已儲存！")

====== 經濟艙模型訓練 ======
📌 RF 測試集 MSE: 0.0422
📌 RF 測試集 R²: 0.8572
📌 SVR 測試集 MSE: 0.0410
📌 SVR 測試集 R²: 0.8612
📌 XGB 測試集 MSE: 0.0402
📌 XGB 測試集 R²: 0.8639

====== 商務艙模型訓練 ======
📌 RF 測試集 MSE: 0.0226
📌 RF 測試集 R²: 0.7568
📌 SVR 測試集 MSE: 0.0233
📌 SVR 測試集 R²: 0.7490
📌 XGB 測試集 MSE: 0.0237
📌 XGB 測試集 R²: 0.7445

✅ 預測完成，合併結果與分別的艙別結果皆已儲存！


## 計算效能指標

In [4]:
import pandas as pd

# 讀取預測結果檔案
economy_file = "/Users/yuchingchen/Documents/專題/model/efficacy_data/short_economy_predicted.csv"
business_file = "/Users/yuchingchen/Documents/專題/model/efficacy_data/short_business_predicted.csv"

economy_class_data = pd.read_csv(economy_file)
business_class_data = pd.read_csv(business_file)

# 定義一個計算效能的函數
def calculate_efficiency(df, pred_col, base_col="隨機購買平均價格"):
    # 計算效能 = (隨機購買平均價格 - 預測價格) / 隨機購買平均價格
    return (df[base_col] - df[pred_col]) / df[base_col]

# 計算經濟艙各模型效能
economy_class_data["效能_rf"] = calculate_efficiency(economy_class_data, "預測最低價格_rf")
economy_class_data["效能_svr"] = calculate_efficiency(economy_class_data, "預測最低價格_svr")
economy_class_data["效能_xgb"] = calculate_efficiency(economy_class_data, "預測最低價格_xgb")

# 計算商務艙各模型效能
business_class_data["效能_rf"] = calculate_efficiency(business_class_data, "預測最低價格_rf")
business_class_data["效能_svr"] = calculate_efficiency(business_class_data, "預測最低價格_svr")
business_class_data["效能_xgb"] = calculate_efficiency(business_class_data, "預測最低價格_xgb")

# 計算平均節省比例（轉換為百分比）
avg_eff_savings_economy_rf = economy_class_data["效能_rf"].mean() * 100
avg_eff_savings_economy_svr = economy_class_data["效能_svr"].mean() * 100
avg_eff_savings_economy_xgb = economy_class_data["效能_xgb"].mean() * 100

avg_eff_savings_business_rf = business_class_data["效能_rf"].mean() * 100
avg_eff_savings_business_svr = business_class_data["效能_svr"].mean() * 100
avg_eff_savings_business_xgb = business_class_data["效能_xgb"].mean() * 100

# 印出結果
print(" 經濟艙平均節省比例:")
print(f"    RF  : {avg_eff_savings_economy_rf:.2f}%")
print(f"    SVR : {avg_eff_savings_economy_svr:.2f}%")
print(f"    XGB : {avg_eff_savings_economy_xgb:.2f}%")

print("\n 商務艙平均節省比例:")
print(f"    RF  : {avg_eff_savings_business_rf:.2f}%")
print(f"    SVR : {avg_eff_savings_business_svr:.2f}%")
print(f"    XGB : {avg_eff_savings_business_xgb:.2f}%")

 經濟艙平均節省比例:
    RF  : 19.45%
    SVR : 19.86%
    XGB : 19.32%

 商務艙平均節省比例:
    RF  : 8.23%
    SVR : 7.85%
    XGB : 8.13%
